In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
import sys
import copy
import statsmodels.api as sm
pd.options.mode.chained_assignment = None
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000
%matplotlib inline

dfTemp=pd.read_csv(r'C:\Users\Jaesung Park\Desktop\FP\GroupProject\final.csv',keep_default_na=False)

In [18]:
dataset = pd.get_dummies(dfTemp, drop_first=True)
dataset = dataset[dataset.columns.difference(['PID','Order','Total Bsmt SF'])]
# mix the dataset
dataset = dataset.reindex(np.random.permutation(dataset.index))


eleven = [] # 11개 빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['MS SubClass_160 ','MS SubClass_20','MS SubClass_30','MS SubClass_50','MS SubClass_60','MS SubClass_70','MS SubClass_other','Lot Area','Street_Pave','Alley_other','Bldg Type_Duplex','Bldg Type_Twnhs','Bldg Type_TwnhsE','Bldg Type_other','FirePlaces','Garage Type_BuiltIn','Garage Type_Detchd','Garage Type_other','Garage Cars','Mo Sold','Yr Sold','Sale Type_other']):
        eleven.append(dataset.columns[i])
        
twentySix = [] # 26빼고 다 넣은 리스트
for i in range(len(dataset.columns)):
    if dataset.columns[i] not in (['MS SubClass_160 ','MS SubClass_20','MS SubClass_30','MS SubClass_50','MS SubClass_60','MS SubClass_70','MS SubClass_other','Lot Area','Street_Pave','Alley_other','Bldg Type_Duplex','Bldg Type_Twnhs','Bldg Type_TwnhsE','Bldg Type_other','FirePlaces','Garage Type_BuiltIn','Garage Type_Detchd','Garage Type_other','Garage Cars','Mo Sold','Yr Sold','Sale Type_other','Lot Shape_Reg','Lot Shape_other','Utilities_other','Lot Config_CulDSac','Lot Config_Inside','Lot Config_other','House Style_1Story','House Style_2Story','House Style_SLvl','House Style_other','Year Built','Year Remod/Add','Roof Style_Hip','Roof Style_other','Foundation_CBlock','Foundation_PConc','Foundation_other','Heating_other','Bedroom AbvGr','Kitchen AbvGr','TotRms AbvGrd','Garage Finish_RFn','Garage Finish_Unf','Garage Finish_other','Misc Feature_other','Sale Condition_Normal','Sale Condition_Partial','Sale Condition_other']):
        twentySix.append(dataset.columns[i])


In [19]:
testing = dataset
# linear Regression
innerRegressor = KNeighborsRegressor(n_neighbors = 10, weights = "distance")
#regressor = LinearRegression(fit_intercept=True)
regressor = LinearRegression(fit_intercept=True)

In [20]:
## exclude the lefts
rounded = round(testing.shape[0]/10)*10
left = testing.shape[0] - rounded
testing2 = testing.iloc[left:]

k = testing2.shape[0]
fold = 10
point = round(k/fold)
mylist = []

start = 0
end= point

# separating dataset into # number of folds and put each fold into list
for i in range(fold):
    x = testing2.iloc[start:end]
    start = start + point
    end = end + point
    mylist.append(x)

In [21]:
RidgeCV_MAE = []
RidgeCV_MSE = []
RidgeCV_RMSE = []
alphasL = []
r2_list = []
predicted = []
temp = copy.deepcopy(mylist)

In [22]:
for j in range(fold):
    mylist = copy.deepcopy(temp)
    testing_set = mylist[j]  
    tempTesting_set = copy.deepcopy(testing_set)
    appended_data = []
    for z in range(fold):
        if not z == j:
            appended_data.append(mylist[z])
    training_set = pd.concat(appended_data, ignore_index = True)
   
    
     
    #Regression imputation
    for p in range(len(testing_set.columns)):
        if testing_set.columns[p] in eleven:
                kfold_testing_X =  testing_set[testing.columns.difference(eleven)]
                kfold_testing_Y =  testing_set[testing_set.columns[p]]
                kfold_training_X = training_set[testing.columns.difference(eleven)]
                kfold_training_Y = training_set[testing_set.columns[p]]
                innerRegressor.fit(kfold_training_X, kfold_training_Y)
                kfold_pred = innerRegressor.predict(kfold_testing_X)   
                testing_set[testing_set.columns[p]] = kfold_pred
    
    testing_set_X = testing_set[testing.columns.difference(['SalePrice'])]
    testing_set_Y = tempTesting_set['SalePrice']
    
    training_set_X = training_set[testing.columns.difference(['SalePrice'])]
    training_set_Y = training_set['SalePrice']

    
    regressor.fit(training_set_X, training_set_Y)
    y_pred = regressor.predict(testing_set_X)
    MAE = metrics.mean_absolute_error(testing_set_Y, y_pred)
    MSE = metrics.mean_squared_error(testing_set_Y, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(testing_set_Y, y_pred))
    r2 = metrics.r2_score(testing_set_Y, y_pred)
    py = y_pred
    
    RidgeCV_MAE.append(MAE)
    RidgeCV_MSE.append(MSE)
    RidgeCV_RMSE.append(RMSE)
    r2_list.append(r2)
    predicted.append(py)


In [23]:
np.mean(RidgeCV_MAE)

48836.52043827997

In [24]:
np.mean(RidgeCV_MSE)

4869597535.948911

In [25]:
np.mean(RidgeCV_RMSE)

69575.07577111066

In [26]:
np.mean(r2_list)

0.23277005696395917